У цьому ДЗ ми потренуємось розв'язувати задачу багатокласової класифікації за допомогою логістичної регресії з використанням стратегій One-vs-Rest та One-vs-One, оцінити якість моделей та порівняти стратегії.

### Опис задачі і даних

**Контекст**

В цьому ДЗ ми працюємо з даними про сегментацію клієнтів.

Сегментація клієнтів – це практика поділу бази клієнтів на групи індивідів, які схожі між собою за певними критеріями, що мають значення для маркетингу, такими як вік, стать, інтереси та звички у витратах.

Компанії, які використовують сегментацію клієнтів, виходять з того, що кожен клієнт є унікальним і що їхні маркетингові зусилля будуть більш ефективними, якщо вони орієнтуватимуться на конкретні, менші групи зі зверненнями, які ці споживачі вважатимуть доречними та які спонукатимуть їх до купівлі. Компанії також сподіваються отримати глибше розуміння уподобань та потреб своїх клієнтів з метою виявлення того, що кожен сегмент цінує найбільше, щоб точніше адаптувати маркетингові матеріали до цього сегменту.

**Зміст**.

Автомобільна компанія планує вийти на нові ринки зі своїми існуючими продуктами (P1, P2, P3, P4 і P5). Після інтенсивного маркетингового дослідження вони дійшли висновку, що поведінка нового ринку схожа на їхній існуючий ринок.

На своєму існуючому ринку команда з продажу класифікувала всіх клієнтів на 4 сегменти (A, B, C, D). Потім вони здійснювали сегментовані звернення та комунікацію з різними сегментами клієнтів. Ця стратегія працювала для них надзвичайно добре. Вони планують використати ту саму стратегію на нових ринках і визначили 2627 нових потенційних клієнтів.

Ви маєте допомогти менеджеру передбачити правильну групу для нових клієнтів.

В цьому ДЗ використовуємо дані `customer_segmentation_train.csv`[скачати дані](https://drive.google.com/file/d/1VU1y2EwaHkVfr5RZ1U4MPWjeflAusK3w/view?usp=sharing). Це `train.csv`з цього [змагання](https://www.kaggle.com/datasets/abisheksudarshan/customer-segmentation/data?select=train.csv)

**Завдання 1.** Завантажте та підготуйте датасет до аналізу. Виконайте обробку пропущених значень та необхідне кодування категоріальних ознак. Розбийте на тренувальну і тестувальну вибірку, де в тесті 20%. Памʼятаємо, що весь препроцесинг ліпше все ж тренувати на тренувальній вибірці і на тестувальній лише використовувати вже натреновані трансформери.
Але в даному випадку оскільки значень в категоріях небагато, можна зробити обробку і на оригінальних даних, а потім розбити - це простіше. Можна також реалізувати процесинг і тренування моделі з пайплайнами. Обирайте як вам зручніше.

In [1]:
from google.colab import files
uploaded = files.upload()

Saving customer_segmentation_train.csv to customer_segmentation_train.csv


In [5]:
import pandas as pd

df = pd.read_csv('customer_segmentation_train.csv')

df.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


In [6]:
print(df.columns.tolist())

['ID', 'Gender', 'Ever_Married', 'Age', 'Graduated', 'Profession', 'Work_Experience', 'Spending_Score', 'Family_Size', 'Var_1', 'Segmentation']


In [9]:
print("Форма датасету:", df.shape)

df.info()

print("\nПропущені значення:\n", df.isnull().sum())

print("\nУнікальні сегменти:", df['Segmentation'].unique())

Форма датасету: (8068, 11)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               8068 non-null   int64  
 1   Gender           8068 non-null   object 
 2   Ever_Married     7928 non-null   object 
 3   Age              8068 non-null   int64  
 4   Graduated        7990 non-null   object 
 5   Profession       7944 non-null   object 
 6   Work_Experience  7239 non-null   float64
 7   Spending_Score   8068 non-null   object 
 8   Family_Size      7733 non-null   float64
 9   Var_1            7992 non-null   object 
 10  Segmentation     8068 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 693.5+ KB

Пропущені значення:
 ID                   0
Gender               0
Ever_Married       140
Age                  0
Graduated           78
Profession         124
Work_Experience    829
Spending_Score       

In [21]:
# 1. Заповнення пропущених значень
for col in ['Ever_Married', 'Graduated', 'Profession', 'Var_1']:
    df[col] = df[col].fillna(df[col].mode()[0])

for col in ['Work_Experience', 'Family_Size']:
    df[col] = df[col].fillna(df[col].median())

In [28]:
# 2. Кодування категоріальних ознак
from sklearn.preprocessing import LabelEncoder

df_encoded = df.copy()
label_encoders = {}

# Кодуємо всі object-колонки, крім цільової 'Segmentation'
for col in df_encoded.columns:
    if df_encoded[col].dtype == 'object' and col != 'Segmentation':
        le = LabelEncoder()
        df_encoded[col] = le.fit_transform(df_encoded[col])
        label_encoders[col] = le

# Кодуємо цільову колонку (Segmentation)
le_target = LabelEncoder()
df_encoded['Segmentation'] = le_target.fit_transform(df_encoded['Segmentation'])

In [29]:
# 3. Виділення X та y
X = df_encoded.drop(['ID', 'Segmentation'], axis=1)
y = df_encoded['Segmentation']

In [30]:
# 4. Розбиття на train/test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)

Train shape: (6454, 9)
Test shape: (1614, 9)


**Завдання 2. Важливо уважно прочитати все формулювання цього завдання до кінця!**

Застосуйте методи ресемплингу даних SMOTE та SMOTE-Tomek з бібліотеки imbalanced-learn до тренувальної вибірки. В результаті у Вас має вийти 2 тренувальних набори: з апсемплингом зі SMOTE, та з ресамплингом з SMOTE-Tomek.

Увага! В нашому наборі даних є як категоріальні дані, так і звичайні числові. Базовий SMOTE не буде правильно працювати з категоріальними даними, але є його модифікація, яка буде. Тому в цього завдання є 2 виконання

  1. Застосувати SMOTE базовий лише на НЕкатегоріальних ознаках.

  2. Переглянути інформацію про метод [SMOTENC](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTENC.html#imblearn.over_sampling.SMOTENC) і використати цей метод в цій задачі. За цей спосіб буде +3 бали за це завдання і він рекомендований для виконання.

  **Підказка**: аби скористатись SMOTENC треба створити змінну, яка містить індекси ознак, які є категоріальними (їх номер серед колонок) і передати при ініціації екземпляра класу `SMOTENC(..., categorical_features=cat_feature_indeces)`.
  
  Ви також можете розглянути варіант використання варіації SMOTE, який працює ЛИШЕ з категоріальними ознаками [SMOTEN](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTEN.html)

In [31]:
!pip install -U imbalanced-learn

In [32]:
from imblearn.over_sampling import SMOTE, SMOTENC
from imblearn.combine import SMOTETomek

In [33]:
# Повертає список назв колонок категоріального типу
cat_columns = X_train.select_dtypes(include='int').columns.tolist()

# Якщо ти кодувала категорії через LabelEncoder, вони всі стали int – тому можна вважати їх "категоріальними"
# Але можна також вказати явно:
cat_columns = ['Gender', 'Ever_Married', 'Graduated', 'Profession', 'Spending_Score', 'Var_1']

# Отримаємо індекси цих колонок у X_train
cat_indices = [X_train.columns.get_loc(col) for col in cat_columns]

In [34]:
smote_tomek = SMOTETomek(random_state=42)
X_resampled_1, y_resampled_1 = smote_tomek.fit_resample(X_train, y_train)

print("Після SMOTE-Tomek:", X_resampled_1.shape)

Після SMOTE-Tomek: (6352, 9)


In [35]:
smotenc = SMOTENC(categorical_features=cat_indices, random_state=42)
X_resampled_2, y_resampled_2 = smotenc.fit_resample(X_train, y_train)

print("Після SMOTENC:", X_resampled_2.shape)

Після SMOTENC: (7256, 9)


**Завдання 3**.
  1. Навчіть модель логістичної регресії з використанням стратегії One-vs-Rest з логістичною регресією на оригінальних даних, збалансованих з SMOTE, збалансованих з Smote-Tomek.  
  2. Виміряйте якість кожної з натренованих моделей використовуючи `sklearn.metrics.classification_report`.
  3. Напишіть, яку метрику ви обрали для порівняння моделей.
  4. Яка модель найкраща?
  5. Якщо немає суттєвої різниці між моделями - напишіть свою гіпотезу, чому?

In [36]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report

In [37]:
model_original = OneVsRestClassifier(LogisticRegression(max_iter=1000, random_state=42))
model_original.fit(X_train, y_train)
y_pred_original = model_original.predict(X_test)

print("Оригінальні дані:")
print(classification_report(y_test, y_pred_original))

Оригінальні дані:
              precision    recall  f1-score   support

           0       0.39      0.39      0.39       394
           1       0.41      0.08      0.14       372
           2       0.47      0.64      0.54       394
           3       0.59      0.79      0.67       454

    accuracy                           0.49      1614
   macro avg       0.46      0.48      0.44      1614
weighted avg       0.47      0.49      0.45      1614



In [38]:
model_tomek = OneVsRestClassifier(LogisticRegression(max_iter=1000, random_state=42))
model_tomek.fit(X_resampled_1, y_resampled_1)
y_pred_tomek = model_tomek.predict(X_test)

print("SMOTE-Tomek:")
print(classification_report(y_test, y_pred_tomek))

SMOTE-Tomek:
              precision    recall  f1-score   support

           0       0.42      0.38      0.39       394
           1       0.35      0.16      0.22       372
           2       0.48      0.62      0.54       394
           3       0.61      0.77      0.68       454

    accuracy                           0.50      1614
   macro avg       0.46      0.48      0.46      1614
weighted avg       0.47      0.50      0.47      1614



In [39]:
model_smotenc = OneVsRestClassifier(LogisticRegression(max_iter=1000, random_state=42))
model_smotenc.fit(X_resampled_2, y_resampled_2)
y_pred_smotenc = model_smotenc.predict(X_test)

print("SMOTENC:")
print(classification_report(y_test, y_pred_smotenc))

SMOTENC:
              precision    recall  f1-score   support

           0       0.42      0.40      0.41       394
           1       0.37      0.14      0.21       372
           2       0.48      0.63      0.54       394
           3       0.62      0.78      0.69       454

    accuracy                           0.50      1614
   macro avg       0.47      0.49      0.46      1614
weighted avg       0.48      0.50      0.47      1614



Висновок:
Яку метрику обрала:
Я обрала macro average F1-score, оскільки вона однаково враховує кожен клас, незалежно від кількості зразків у класі.


Моделі з SMOTE-Tomek і SMOTENC показали однакові результати (F1 macro = 0.46), трохи кращі за оригінальну модель (F1 macro = 0.44). Отже, обидві моделі з ресемплінгом кращі за оригінал.


Моя гіпотеза: набір даних був не надто сильно розбалансованим, або логістична регресія як лінійна модель має обмежену здатність покращити результат навіть на збалансованих даних. Можливо, сильніше покращення дало б застосування більш складної моделі.

